In [1]:
import h5py
import numpy as np
from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


In [9]:
#process digitStruct.mat according to see_bboxes.m provided in the dataset

path = 'train/train/digitStruct.mat'
file = h5py.File(path, 'r')
#list(file.keys())
#list(file['digitStruct'].keys())
size = len(file['digitStruct']['bbox']) 
size

33402

In [10]:
#tool to navigate the digitStruct file
#idx 1 two digits
#idx 34 one digit
idx = 34
item = file['digitStruct']['bbox'][idx].item()  
#list(file[item].keys())
#['height', 'label', 'left', 'top', 'width']
attr = 'top'
num = len(file[item][attr])
value = []
#file[file[item]['height'].value[0].item()].value
if num == 1:
    loc = file[item][attr]
    value.append(loc.value[0][0])
else:
    for i in range(num):
        loc = file[item][attr].value[i].item()
        value.append(file[loc].value[0][0])
value

[5.0]

In [11]:
height = []
width = []
left = []
top = []
label = []
imgsrc = []
attrs = ['height', 'label', 'left', 'top', 'width']
dataArrays = [height, label, left, top, width]
numDigits = np.zeros(size, dtype = int)

for idx in range(size):
    item = file['digitStruct']['bbox'][idx].item()
    #just read num of digits in 1 image, attr not important here
    attr = 'top'
    num = len(file[item][attr])
    numDigits[idx] = num
    
    for i in range(num):
        imgsrc.append(idx+1)
        
    for attrIdx in range(5):
        attr = attrs[attrIdx]
        dataArray = dataArrays[attrIdx]
        
        if num == 1:
            loc = file[item][attr]
            dataArray.append(loc.value[0][0])
        else:
            value
            for i in range(num):
                loc = file[item][attr].value[i].item()
                dataArray.append(file[loc].value[0][0])
        #..


In [12]:
#save as npy
data = np.zeros((5,len(height)))
data[0] = np.array(height)
data[1] = np.array(width)
data[2] = np.array(left)
data[3] = np.array(top)
data[4] = np.array(imgsrc)
np.save('bbox_data.npy', data)


In [13]:
#convert label format to model output and save them separately

#first make 2d array for labels, each has at most 5 digits (digit 10 means invalid)
labelArray = np.zeros((size, int(np.max(numDigits))), dtype=int) + 10
digitIdx = 0
for i in range(1,size+1):
    count = 0
    while (digitIdx < len(label)) and (int(imgsrc[digitIdx]) == i):
        labelArray[i-1,count] = label[digitIdx]
        digitIdx += 1
        count += 1
labelArray

array([[ 1,  9, 10, 10, 10, 10],
       [ 2,  3, 10, 10, 10, 10],
       [ 2,  5, 10, 10, 10, 10],
       ..., 
       [ 9,  5, 10, 10, 10, 10],
       [ 2,  2, 10, 10, 10, 10],
       [ 1,  6,  9, 10, 10, 10]])

In [14]:
#represent no digit as 10 (categorical), possible value of digit(11)
#0-5 and more than 5, possible value of length(7) 7+11*5
values = np.zeros((size, 7+11*5), dtype=int)
values[:,:7] = to_categorical(np.minimum(numDigits,6), 7)
values[:,7:18] = to_categorical(labelArray[:,0], 11)
values[:,18:29] = to_categorical(labelArray[:,1], 11)
values[:,29:40] = to_categorical(labelArray[:,2], 11)
values[:,40:51] = to_categorical(labelArray[:,3], 11)
values[:,51:62] = to_categorical(labelArray[:,4], 11)
    

np.save('labels.npy', values)
values

array([[0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       ..., 
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [4]:
testpath = 'test/test/digitStruct.mat'
file = h5py.File(testpath, 'r')
size = len(file['digitStruct']['bbox']) 

In [5]:
#tool to navigate the digitStruct file
idx = 34
item = file['digitStruct']['bbox'][idx].item()  
#list(file[item].keys())
#['height', 'label', 'left', 'top', 'width']
attr = 'top'
num = len(file[item][attr])
value = []
#file[file[item]['height'].value[0].item()].value
if num == 1:
    loc = file[item][attr]
    value.append(loc.value[0][0])
else:
    for i in range(num):
        loc = file[item][attr].value[i].item()
        value.append(file[loc].value[0][0])
value

[13.0, 13.0]

In [6]:
height = []
width = []
left = []
top = []
label = []
imgsrc = []
attrs = ['height', 'label', 'left', 'top', 'width']
dataArrays = [height, label, left, top, width]
numDigits = np.zeros(size, dtype = int)

for idx in range(size):
    item = file['digitStruct']['bbox'][idx].item()
    #just read num of digits in 1 image, attr not important here
    attr = 'top'
    num = len(file[item][attr])
    numDigits[idx] = num
    
    for i in range(num):
        imgsrc.append(idx+1)
        
    for attrIdx in range(5):
        attr = attrs[attrIdx]
        dataArray = dataArrays[attrIdx]
        
        if num == 1:
            loc = file[item][attr]
            dataArray.append(loc.value[0][0])
        else:
            value
            for i in range(num):
                loc = file[item][attr].value[i].item()
                dataArray.append(file[loc].value[0][0])
        #..


In [7]:
#save as npy
data = np.zeros((5,len(height)))
data[0] = np.array(height)
data[1] = np.array(width)
data[2] = np.array(left)
data[3] = np.array(top)
data[4] = np.array(imgsrc)
np.save('test_bbox_data.npy', data)


In [8]:
#convert label format to model output and save them separately

#first make 2d array for labels, each has at most 5 digits (digit 10 means invalid)
labelArray = np.zeros((size, int(np.max(numDigits))), dtype=int) + 10
digitIdx = 0
for i in range(1,size+1):
    count = 0
    while (digitIdx < len(label)) and (int(imgsrc[digitIdx]) == i):
        labelArray[i-1,count] = label[digitIdx]
        digitIdx += 1
        count += 1
labelArray

array([[ 5, 10, 10, 10, 10],
       [ 2,  1, 10, 10, 10],
       [ 6, 10, 10, 10, 10],
       ..., 
       [ 1,  3,  6, 10, 10],
       [ 2,  2,  7, 10, 10],
       [ 6,  7, 10, 10, 10]])

In [9]:
#represent no digit as 10 (categorical), possible value of digit(11)
#0-5 and more than 5, possible value of length(7) 7+11*5
values = np.zeros((size, 7+11*5), dtype=int)
values[:,:7] = to_categorical(np.minimum(numDigits,6), 7)
values[:,7:18] = to_categorical(labelArray[:,0], 11)
values[:,18:29] = to_categorical(labelArray[:,1], 11)
values[:,29:40] = to_categorical(labelArray[:,2], 11)
values[:,40:51] = to_categorical(labelArray[:,3], 11)
values[:,51:62] = to_categorical(labelArray[:,4], 11)
    

np.save('test_labels.npy', values)
values

array([[0, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       ..., 
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1]])